In [31]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("snehaanbhawal/resume-dataset")

import os
paths = [
    "/kaggle/input/resume-dataset/Resume/Resume.csv",
    "/root/.cache/kagglehub/datasets/snehaanbhawal/resume-dataset/versions/1/Resume/Resume.csv"
]

for p in paths:
    try:
        df = pd.read_csv(p, encoding="latin-1")
        print("Successfully loaded:", p)
        break
    except:
        pass


Using Colab cache for faster access to the 'resume-dataset' dataset.
Successfully loaded: /kaggle/input/resume-dataset/Resume/Resume.csv


In [32]:
from google.colab import drive
drive.mount('/content/drive')
import os
print(os.listdir('/content/drive/MyDrive'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Colab Notebooks', 'aurora.zip', 'interested jobs.gsheet', 'Critical Analysis Content Summary #3.gdoc', 'Items to sell.gsheet', 'ONET']


In [33]:
import os
print(os.listdir('/content/drive/MyDrive/ONET'))

['Job Zone Reference.txt', 'Tasks to DWAs.txt', 'Knowledge.txt', 'Content Model Reference.txt', 'Work Styles.txt', 'Emerging Tasks.txt', 'Abilities.txt', 'Abilities to Work Context.txt', 'Level Scale Anchors.txt', 'Related Occupations.txt', 'Interests.txt', 'RIASEC Keywords.txt', 'Survey Booklet Locations.txt', 'Scales Reference.txt', 'Read Me.txt', 'Education, Training, and Experience.txt', 'Abilities to Work Activities.txt', 'IWA Reference.txt', 'UNSPSC Reference.txt', 'Work Context Categories.txt', 'Job Zones.txt', 'Task Categories.txt', 'Education, Training, and Experience Categories.txt', 'Sample of Reported Titles.txt', 'Skills to Work Context.txt', 'Basic Interests to RIASEC.txt', 'Work Values.txt', 'Occupation Data.txt', 'Interests Illustrative Activities.txt', 'Skills to Work Activities.txt', 'DWA Reference.txt', 'Interests Illustrative Occupations.txt', 'Technology Skills.txt', 'Skills.txt', 'Task Ratings.txt', 'Task Statements.txt', 'Work Activities.txt', 'Occupation Level M

In [34]:
import pandas as pd

skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
abilities = pd.read_csv('/content/drive/MyDrive/ONET/Abilities.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
occupation = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')

In [35]:
import pandas as pd

# Loading key ONET files
occ = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
activities = pd.read_csv('/content/drive/MyDrive/ONET/Work Activities.txt', sep='\t', encoding='latin-1')

# Group aggregation (join multiple rows per occupation)
occ_skills = skills.groupby('O*NET-SOC Code')['Element Name'].apply(list).reset_index()
occ_tasks = tasks.groupby('O*NET-SOC Code')['Task'].apply(list).reset_index()
occ_activities = activities.groupby('O*NET-SOC Code')['Element Name'].apply(list).reset_index()

# Merge them all
occupation_profile = occ[['O*NET-SOC Code', 'Title', 'Description']] \
    .merge(occ_skills, on='O*NET-SOC Code', how='left') \
    .merge(occ_tasks, on='O*NET-SOC Code', how='left') \
    .merge(occ_activities, on='O*NET-SOC Code', how='left')

occupation_profile.head()

,O*NET-SOC Code,Title,Description,Element Name_x,Task,Element Name_y
0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,"[Reading Comprehension, Reading Comprehension,...",[Direct or coordinate an organization's financ...,"[Getting Information, Getting Information, Mon..."
1,11-1011.03,Chief Sustainability Officers,"Communicate and coordinate with management, sh...","[Reading Comprehension, Reading Comprehension,...",[Monitor and evaluate effectiveness of sustain...,"[Getting Information, Getting Information, Mon..."
2,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of ...","[Reading Comprehension, Reading Comprehension,...","[Review financial statements, sales or activit...","[Getting Information, Getting Information, Mon..."
3,11-1031.00,Legislators,"Develop, introduce, or enact laws and statutes...",NaN,[Analyze and understand the local and national...,NaN
4,11-2011.00,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici...","[Reading Comprehension, Reading Comprehension,...",[Plan and prepare advertising and promotional ...,"[Getting Information, Getting Information, Mon..."


In [36]:
!pip install sentence-transformers

In [37]:
print(occupation_profile.columns)
occupation_profile = occupation_profile.rename(columns={
    'Element Name_x': 'Skills',
    'Task': 'Tasks',
    'Element Name_y': 'Work_Activities'
})
for col in ['Skills', 'Tasks', 'Work_Activities']:
    occupation_profile[col] = occupation_profile[col].apply(
        lambda x: list(set(x)) if isinstance(x, list) else x
    )
def combine_text(row):
    text = f"{row['Title']} {row['Description']} "
    if isinstance(row['Skills'], list):
        text += ' '.join(row['Skills'])
    if isinstance(row['Tasks'], list):
        text += ' '.join(row['Tasks'])
    if isinstance(row['Work_Activities'], list):
        text += ' '.join(row['Work_Activities'])
    return text
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
occupation_profile['combined_text'] = occupation_profile.apply(combine_text, axis=1)
occupation_profile['embedding'] = occupation_profile['combined_text'].apply(lambda x: model.encode(str(x)))


Index(['O*NET-SOC Code', 'Title', 'Description', 'Element Name_x', 'Task',
       'Element Name_y'],
      dtype='object')


In [38]:
def explain_match(resume_skills, job_skills):
    if isinstance(job_skills, list):
        return list(set([s.lower() for s in job_skills]) & set(resume_skills))
    return []

In [43]:
211import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Resume dataset
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
print(resumes.columns)  # Check actual column names

# Example: if resume text column is named 'Resume', otherwise adjust:
resume_text_column = 'Resume' if 'Resume' in resumes.columns else resumes.columns[0]

# O*NET dataset
onet_path = '/content/drive/MyDrive/ONET/Task Statements.txt'
tasks = pd.read_csv(onet_path, sep='\t', encoding='latin-1')
print(tasks.columns)  # Check what columns exist

Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
Index(['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type',
       'Incumbents Responding', 'Date', 'Domain Source'],
      dtype='object')


In [47]:
# Install required libraries
!pip install sentence-transformers --quiet

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ------------------------------
# 1. Load Datasets
# ------------------------------

# Resume dataset (Kaggle)
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
print("Resume columns:", resumes.columns)

# Assume first column contains resume text
resume_text_col = 'Resume' if 'Resume' in resumes.columns else resumes.columns[0]

# O*NET occupation dataset (adjust path in Colab)
# Combine relevant fields later into 'combined_text'
onet_path = '/content/drive/MyDrive/ONET/Task Statements.txt'
tasks = pd.read_csv(onet_path, sep='\t', encoding='latin-1')
print("O*NET columns:", tasks.columns)

# For demonstration, let's load Skills.txt as well
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')

# Merge or create an occupation profile (simplified)
occupation_profile = skills.copy()
occupation_profile['combined_text'] = occupation_profile['Element Name'].astype(str)  # Use 'Element Name' or combine multiple columns

# ------------------------------
# 2. Initialize embedding model
# ------------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

# ------------------------------
# 3. Generate embeddings
# ------------------------------
# Resumes
resumes['embedding'] = resumes[resume_text_col].astype(str).apply(lambda x: model.encode(x))

# Occupations
occupation_profile['embedding'] = occupation_profile['combined_text'].astype(str).apply(lambda x: model.encode(x))

# ------------------------------
# 4. Compute similarity and recommend top jobs
# ------------------------------
def recommend_jobs(resume_embedding, occupation_embeddings, occupation_titles, top_k=5):
    sims = cosine_similarity([resume_embedding], list(occupation_embeddings))[0]
    top_idx = sims.argsort()[::-1][:top_k]
    top_jobs = occupation_titles.iloc[top_idx]
    top_scores = sims[top_idx]
    return pd.DataFrame({'Job Title': top_jobs, 'Similarity': top_scores})

# Example: Recommend for first resume
first_resume_embedding = resumes['embedding'][0]
recommendations = recommend_jobs(first_resume_embedding,
                                 occupation_profile['embedding'],
                                 occupation_profile['Element Name'],
                                 top_k=5)

print("Top 5 job recommendations for first resume:")
print(recommendations)

Resume columns: Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
O*NET columns: Index(['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type',
       'Incumbents Responding', 'Date', 'Domain Source'],
      dtype='object')
Top 5 job recommendations for first resume:
             Job Title  Similarity
50     Troubleshooting    0.211162
62560  Troubleshooting    0.211162
38691  Troubleshooting    0.211162
29241  Troubleshooting    0.211162
58920  Troubleshooting    0.211162


In [ ]:
# ------------------------------
# 1. Install required library
# ------------------------------
!pip install sentence-transformers --quiet
!pip install umap-learn --quiet

# ------------------------------
# 2. Imports
# ------------------------------
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import re
import umap

# ------------------------------
# 3. Load datasets
# ------------------------------
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
resume_text_col = 'Resume_str'

# O*NET datasets
occupation_data = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
work_activities = pd.read_csv('/content/drive/MyDrive/ONET/Work Activities.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
knowledge = pd.read_csv('/content/drive/MyDrive/ONET/Knowledge.txt', sep='\t', encoding='latin-1')
work_context = pd.read_csv('/content/drive/MyDrive/ONET/Work Context.txt', sep='\t', encoding='latin-1')

# ------------------------------
# 4. Preprocess resumes
# ------------------------------
def clean_resume(text):
    text = re.sub(r'<.*?>', ' ', str(text))  # remove HTML
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

resumes['cleaned_resume'] = resumes[resume_text_col].apply(clean_resume)

# ------------------------------
# 5. Aggregate text per occupation
# ------------------------------
def aggregate_text(df, group_col='O*NET-SOC Code', text_col='Element Name', new_col_name=None):
    agg = df.groupby(group_col)[text_col].apply(lambda x: ' '.join(x.astype(str))).reset_index()
    if new_col_name:
        agg = agg.rename(columns={text_col: new_col_name})
    return agg

skills_agg = aggregate_text(skills, text_col='Element Name', new_col_name='Skills')
work_agg = aggregate_text(work_activities, text_col='Element Name', new_col_name='Work_Activities')
tasks_agg = aggregate_text(tasks, text_col='Task', new_col_name='Tasks')
knowledge_agg = aggregate_text(knowledge, text_col='Element Name', new_col_name='Knowledge')
context_agg = aggregate_text(work_context, text_col='Element Name', new_col_name='Work_Context')

# ------------------------------
# 6. Merge into occupation profile
# ------------------------------
occupation_profile = occupation_data.merge(skills_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(work_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(tasks_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(knowledge_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(context_agg, on='O*NET-SOC Code', how='left')

# ------------------------------
# 7. Combine text fields (weighted)
# ------------------------------
occupation_profile['combined_text'] = (
    occupation_profile['Title'].astype(str) + ' ' +
    occupation_profile['Description'].astype(str) + ' ' +
    3*occupation_profile['Skills'].astype(str) + ' ' +       # weight skills higher
    2*occupation_profile['Tasks'].astype(str) + ' ' +       # weight tasks moderately
    occupation_profile['Work_Activities'].astype(str) + ' ' +
    occupation_profile['Knowledge'].astype(str) + ' ' +
    occupation_profile['Work_Context'].astype(str)
)

# ------------------------------
# 8. Initialize embedding model (higher quality)
# ------------------------------
model = SentenceTransformer('all-mpnet-base-v2')

# ------------------------------
# 9. Generate embeddings
# ------------------------------
print("Generating occupation embeddings...")
occupation_profile['embedding'] = occupation_profile['combined_text'].apply(lambda x: model.encode(str(x)))

print("Generating resume embeddings...")
resumes['embedding'] = resumes['cleaned_resume'].apply(lambda x: model.encode(str(x)))

# Optional: dimensionality reduction (UMAP) to speed up similarity search if dataset is huge
embeddings_matrix = list(occupation_profile['embedding'])
umap_model = umap.UMAP(n_components=256, random_state=42)
occupation_profile['embedding_reduced'] = list(umap_model.fit_transform(embeddings_matrix))

# ------------------------------
# 10. Recommendation function
# ------------------------------
def recommend_jobs(resume_embedding, occupation_embeddings, occupation_titles, top_k=5):
    sims = cosine_similarity([resume_embedding], list(occupation_embeddings))[0]
    top_idx = sims.argsort()[::-1][:top_k]
    return pd.DataFrame({'Job Title': occupation_titles.iloc[top_idx], 'Similarity': sims[top_idx]})

# ------------------------------
# 11. Example: top 5 recommendations for first resume
# ------------------------------
first_resume_embedding = resumes['embedding'][0]
recommendations = recommend_jobs(first_resume_embedding,
                                 occupation_profile['embedding'],
                                 occupation_profile['Title'],
                                 top_k=5)
print("Top 5 job recommendations for first resume:")
print(recommendations)

# ------------------------------
# 12. Loop over all resumes and save CSV
# ------------------------------
all_recommendations = []
for idx, row in resumes.iterrows():
    recs = recommend_jobs(row['embedding'], occupation_profile['embedding'], occupation_profile['Title'], top_k=5)
    recs['Resume_ID'] = row['ID']
    all_recommendations.append(recs)

final_recs = pd.concat(all_recommendations, ignore_index=True)
final_recs.to_csv('resume_job_recommendations_final.csv', index=False)
print("Saved all top 5 job recommendations to 'resume_job_recommendations_final.csv'.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating occupation embeddings...
Generating resume embeddings...
